### Prototype of integrating TVB with SIIBRA - Download a structural connectivity from SIIBRA and setup a TVB simulation with it
#### (This is an adaptation of https://github.com/dickscheid/siibra-tutorials/blob/main/06-SIIBRA-TVB.ipynb, which used an older version of siibra-python)

### Imports and setup

In [ ]:
%matplotlib widget

In [ ]:
import os
import numpy as np
import siibra
from tvb.simulator.lab import *

### Make sure an EBRAINS token exists

In [ ]:
if not 'HBP_AUTH_TOKEN' in os.environ:
    print("Missing auth token for siibra!!")
    
#os.environ['HBP_AUTH_TOKEN'] = ''
# alterantively, use siibra service for authentication into EBRAINS
# siibra.fetch_ebrains_token()

### Get connectivity matrices

#### Connectivity weights

In [ ]:
# get desired brain parcellation
atlas = siibra.atlases["human"]
jubrain = atlas.get_parcellation("julich 2.9")

In [ ]:
# get the Streamline Counts (conn. weights) features
# this gives a list of Streamline Counts objects corresponding to a different cohort; each Streamline Count obj. has multiple connectivities
features = siibra.features.get(jubrain, siibra.features.connectivity.StreamlineCounts)
for f in features:
    print(f.cohort)

In [ ]:
# select one of the cohorts and get all the connectivities for all the subjects
conn_weights = features[0]
print(f'There are connectivity weights available for {len(conn_weights.subjects)} subjects')

In [ ]:
# get the connectivity matrix for one of the subjects
subject = conn_weights.subjects[0]  # this gives us a string containing the subject id
weights_matrix = conn_weights.get_matrix(subject)  # matrix stored as pandas DataFrame
weights_matrix

#### Connectivity tracts

In [ ]:
# get the Streamline Lengths (conn. tracts) features
# this gives a list of Streamline Lengths object corresponding to a different cohort; each Streamline Lengths obj. has multiple connectivities
features_lengths = siibra.features.get(jubrain, siibra.features.connectivity.StreamlineLengths)
for f in features_lengths:
    print(f.cohort)

In [ ]:
# select one of the cohorts and get all the connectivities for all the subjects
conn_tracts = features_lengths[0]
conn_tracts.name
print(f'There are connectivity tracts available for {len(conn_tracts.subjects)} subjects')

In [ ]:
# get the connectivity matrix for one of the subjects
subject = conn_tracts.subjects[0]  # this gives us a string containing the subject id
tracts_matrix = conn_tracts.get_matrix(subject)  # matrix stored as pandas DataFrame
tracts_matrix[10:20]

In [ ]:
# check that the weights and tracts have the same format
assert len(weights_matrix) == len(tracts_matrix)
assert weights_matrix.columns.to_list() == tracts_matrix.columns.to_list()
assert (weights_matrix.index == tracts_matrix.index).all()

### Get region names
##### (In siibra the indices of weights/tracts matrix are Region objects*)
\* sometimes they are tuples, where first value is name of parent of that region and second value is the actual region

In [ ]:
# get list of region objects
regions = weights_matrix.index.values
# because sometimes we have tuples instead of regions, correct the list to have only regions
regions = [r[1] if type(r)==tuple else r for r in regions]
regions

In [ ]:
# get region names and the corresponding hemispheres
reg_names = []
hemi = []
for r in regions:
    name = r.name
    reg_names.append(name)
    
    if 'right' in name:
        hemi.append(1)
    # there is a bug on the else branch: there are regions which refer to both the right and left hemishperes;
    # right now they are put in the left hemisphere, but this is wrong! and should be corrected in some way
    else:
        hemi.append(0)

In [ ]:
reg_names[:5]

In [ ]:
# check the correctness of hemi array
hemi[:5]

In [ ]:
# save regions related to both hemispheres for future reference?
both_hemi_regions = []
for r in regions:
    name = r.name
    if 'left' not in name and 'right' not in name:
        both_hemi_regions.append(r)
both_hemi_regions

### Get region positions

In [ ]:
# first we need a space in which the positions are computed
space = atlas.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC # commonly used space in other examples

In [ ]:
r1 = regions[0]

In [ ]:
tuple(r1.spatial_props(space)['components'][0]['centroid'])

In [ ]:
positions = []
for r in regions:
    spatial_props = r.spatial_props(space) # gives a dict of spatial props
    # get centroids list
    centroids = spatial_props['components']
    # get siibra.Point object from centroid list; some regions have multiple centroids, but only the first one is selected
    centroids = centroids[0]['centroid']
    # tuple() gives the coordinates of a centroid
    positions.append(tuple(centroids))
positions

### Create TVB Connectivity with data obtained using siibra

In [ ]:
# Moving info from Siibra into TVB concepts
conn = connectivity.Connectivity()
conn.weights = weights_matrix.to_numpy()
conn.tract_lengths = tracts_matrix.to_numpy()
conn.region_labels = np.array(reg_names)
conn.hemispheres = np.array(hemi, dtype=np.bool_)
conn.centres = np.array(positions)

conn.configure()
conn

In [ ]:
plot_connectivity(connectivity=conn)

In [ ]:
# Save connectivity in TVB accepted format
centres_content = np.concatenate((np.array(reg_names)[:, None], positions), axis=1)


root_folder = 'julich_conn'
out_dir = os.path.join(root_folder, subject)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

np.savetxt(os.path.join(out_dir, "centers.txt"), centres_content, "%s")
np.savetxt(os.path.join(out_dir, "hemispheres.txt"), hemi, "%s")
np.savetxt(os.path.join(out_dir, "weights.txt"), weights_matrix.to_numpy(), "%f")
np.savetxt(os.path.join(out_dir, "tract_lenghts.txt"), tracts_matrix.to_numpy(), "%f")

### Create simulation using the obtained connectivity

In [ ]:
sim = simulator.Simulator()
sim.connectivity = conn
sim.simulation_length = 1024
sim.configure()

In [ ]:
(time, data),  = sim.run()

In [ ]:
time.size

In [ ]:
data.shape

In [ ]:
tsr = time_series.TimeSeriesRegion(
    data=data,
    connectivity=sim.connectivity,
    sample_period=sim.monitors[0].period)
tsr.configure()

In [ ]:
import tvb.simulator.plot.timeseries_interactive as ts_int
tsi = ts_int.TimeSeriesInteractive(time_series=tsr)
tsi.configure()
tsi.show()